In [1]:
#! /usr/bin/env python
"""
Pre-process temperature data for missing files
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

debug=False

In [2]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [3]:
# Mosaic multiple files
#  Note: for large/difficult regions, run this in batches to mosaic each batch (other script accepts multiple files)
mosaic_ts_info = False
from rasterio.merge import merge

# ====== MERGE CALIBRATED GLACIERS INTO DEGREE PIXELS ======
# Export all .tifs to a common projection that will enable merging them smoothly
dst_crs = 'EPSG:4326'

ts_fp = debris_prms.ts_fp + 'missing/'

# All Ts filenames
ts_fullfns = []
ts_year_fullfns = []
ts_doy_fullfns = []
ts_dayfrac_fullfns = []
for i in os.listdir(ts_fp):
    if i.endswith('.tif'):
        if 'tsurfC' in i:
            ts_fullfns.append(ts_fp + i)
        elif 'year' in i:
            ts_year_fullfns.append(ts_fp + i)
        elif 'doy' in i:
            ts_doy_fullfns.append(ts_fp + i)
        elif 'dayfrac' in i:
            ts_dayfrac_fullfns.append(ts_fp + i)

if mosaic_ts_info:
    
    # All Ts filenames
    ts_fullfns = []
    ts_year_fullfns = []
    ts_doy_fullfns = []
    ts_dayfrac_fullfns = []
    for i in os.listdir(ts_fp):
        if i.endswith('.tif'):
            if 'tsurfC' in i:
                ts_fullfns.append(ts_fp + i)
            elif 'year' in i:
                ts_year_fullfns.append(ts_fp + i)
            elif 'doy' in i:
                ts_doy_fullfns.append(ts_fp + i)
            elif 'dayfrac' in i:
                ts_dayfrac_fullfns.append(ts_fp + i)

    ts_fullfns = sorted(ts_fullfns)
    ts_year_fullfns = sorted(ts_year_fullfns)
    ts_doy_fullfns = sorted(ts_doy_fullfns)
    ts_dayfrac_fullfns = sorted(ts_dayfrac_fullfns)
    
    if debug:
        print(ts_fullfns)
        print(ts_year_fullfns)
        print(ts_doy_fullfns)
        print(ts_dayfrac_fullfns)
        
    

    def mosaic_fullfns(list_fullfns, fn_merged=None):
        if fn_merged is None:
            # Hack to deal with naming convention I used of '16-03_...' in GEE
            if debris_prms.roi not in ['10', '16']:
                fn_merged = list_fullfns[0].split('/')[-1].split('-')[0] + '.tif'
            else:
                fn_merged = (list_fullfns[0].split('/')[-1].split('-')[0] + '-' + 
                             list_fullfns[0].split('/')[-1].split('-')[1] + '.tif')
        src_files_to_mosaic = []
        for fn in list_fullfns:
            with rasterio.open(fn) as src:
                src = rasterio.open(fn)
                src_files_to_mosaic.append(src)

        mos, out_trans = merge(src_files_to_mosaic)

        with rasterio.open(ts_fp + fn_merged,"w", driver ='Gtiff',count=1,
                           height= mos.shape[1],
                           width= mos.shape[2],
                           transform= out_trans,
                           crs= src.crs,
                           dtype= src.dtypes[0]) as dest:
            dest.write(mos)
            
    if len(ts_fullfns) > 1:
        mosaic_fullfns(ts_fullfns)
    if len(ts_year_fullfns) > 1:
        mosaic_fullfns(ts_year_fullfns)
    if len(ts_doy_fullfns) > 1:
        mosaic_fullfns(ts_doy_fullfns)
    if len(ts_dayfrac_fullfns) > 1:
        mosaic_fullfns(ts_dayfrac_fullfns)

In [4]:
# ===== LOAD ALL GLACIERS =====
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

dc_shp.reset_index(inplace=True, drop=True)
dc_shp['CenLon_360'] = dc_shp['CenLon']
dc_shp.loc[dc_shp['CenLon_360'] < 0, 'CenLon_360'] = 360 + dc_shp.loc[dc_shp['CenLon_360'] < 0, 'CenLon_360']

rgiid_list = [x.split('-')[1] for x in dc_shp['RGIId'].values]
main_glac_rgi_all = debris_prms.selectglaciersrgitable(rgiid_list)

# ===== DETERMINE ALL GLACIERS WITH AND WITHOUT OBSERVATIONS =====
hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + debris_prms.roi + '/'

# Glaciers optimized
glac_tsopt_fns = []
rgiid_list_tsopt = []
for roi_extrap in debris_prms.roi_dict_extrap[debris_prms.roi]:
    hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + roi_extrap + '/'
    for i in os.listdir(hdopt_prms_fp):
        if i.endswith('_hdopt_prms.csv'):
            region = int(i.split('.')[0])
            if region in debris_prms.roi_rgidict[roi_extrap]:    
                rgiid_list_tsopt.append(i.split('_')[0])            
                glac_tsopt_fns.append(i)
    
    for i in os.listdir(hdopt_prms_fp + '_extrap/'):
        if i.endswith('_hdopt_prms_extrap.csv'):
            region = int(i.split('.')[0])
            if region in debris_prms.roi_rgidict[roi_extrap]:    
                rgiid_list_tsopt.append(i.split('_')[0])            
                glac_tsopt_fns.append(i)

glac_tsopt_fns = sorted(glac_tsopt_fns)
rgiid_list_tsopt = sorted(rgiid_list_tsopt)

main_glac_rgi_tsopt = debris_prms.selectglaciersrgitable(rgiid_list_tsopt)

# Glaciers lacking optimization
rgiids_missing = set(main_glac_rgi_all.rgino_str.values) - set(main_glac_rgi_tsopt.rgino_str.values)
rgiids_missing = sorted(rgiids_missing)
main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)
main_glac_rgi_missing

1965 glaciers in region 3 are included in this model run: ['00093', '00094', '00095', '00096', '00097', '00098', '00099', '00100', '00102', '00103', '00104', '00105', '00106', '00107', '00108', '00109', '00110', '00113', '00114', '00115', '00116', '00117', '00118', '00120', '00123', '00124', '00125', '00126', '00128', '00129', '00130', '00131', '00132', '00133', '00134', '00135', '00136', '00137', '00139', '00140', '00141', '00142', '00143', '00144', '00145', '00146', '00147', '00148', '00150', '00151'] and more
This study is focusing on 1965 glaciers in region [3]
1816 glaciers in region 3 are included in this model run: ['00093', '00094', '00095', '00096', '00097', '00098', '00099', '00100', '00102', '00103', '00104', '00105', '00106', '00107', '00109', '00110', '00114', '00115', '00116', '00117', '00118', '00120', '00124', '00125', '00126', '00128', '00129', '00130', '00131', '00132', '00133', '00134', '00135', '00136', '00137', '00139', '00140', '00141', '00142', '00143', '00144', 

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,107,RGI60-03.00108,-74.7935,80.7189,3,3,490.205,0,2024,1582,6.2,313,64821,1,1,1,20019999,108,03.00108,3.00108
1,112,RGI60-03.00113,-75.3588,80.5373,3,3,999.861,0,2074,1446,6.3,323,71606,1,1,0,20019999,113,03.00113,3.00113
2,122,RGI60-03.00123,-76.5299,80.3547,3,3,1594.482,0,2106,1438,5.1,311,74478,0,1,2,20019999,123,03.00123,3.00123
3,174,RGI60-03.00175,-79.9514,79.8193,3,3,154.181,0,1392,994,8.5,205,31922,1,1,0,20019999,175,03.00175,3.00175
4,182,RGI60-03.00183,-78.4826,79.9435,3,3,940.826,0,2021,1439,6.8,196,73045,1,1,0,20019999,183,03.00183,3.00183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,3987,RGI60-03.03989,-84.7807,82.0769,3,1,36.079,0,1355,795,10.2,174,2776,0,1,2,20019999,3989,03.03989,3.03989
145,3990,RGI60-03.03992,-86.2945,82.1364,3,1,19.009,0,1130,514,12.2,208,6322,0,1,0,20019999,3992,03.03992,3.03992
146,4379,RGI60-03.04381,-84.8897,80.9541,3,1,53.547,0,1147,916,9.6,309,8416,0,1,0,20019999,4381,03.04381,3.04381


In [5]:
add_ts_fns = True
if add_ts_fns:

    # Ts filenames dataframe
    if os.path.exists(debris_prms.ts_fp + debris_prms.ts_fns_fn):
        ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
        
    if debug:
        print(ts_fullfns)
        print(ts_year_fullfns)
        print(ts_doy_fullfns)
        print(ts_dayfrac_fullfns)
    
    # ===== PROCESS EACH GLACIER =====
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
        glac_str = main_glac_rgi_missing.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_missing.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'

        ts_fn_wglacier = None
        ts_year_fn_wglacier = None
        ts_doy_fn_wglacier = None
        ts_dayfrac_fn_wglacier = None
        
        if os.path.exists(thick_dir + thick_fn):

            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # ==== CHECK IF TIF HAS SURFACE TEMPERATURE DATA OVER THE GLACIER =====
            # Add the filenames
            fn_dict = OrderedDict()
            # DEM
            z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
            z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
            fn_dict['z1'] = z1_fp + z1_fn
            # Ice thickness
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            fn_dict['ice_thick'] = thick_dir + thick_fn
            # Ts
            ts_dict_keys = []
            for nfn, ts_fn in enumerate(ts_fullfns):
                fn_dict['ts_' + str(nfn)] = ts_fn
                ts_dict_keys.append('ts_' + str(nfn))
            ts_year_dict_keys = []
            for nfn, ts_year_fn in enumerate(ts_year_fullfns):
                fn_dict['ts_year_' + str(nfn)] = ts_year_fn
                ts_year_dict_keys.append('ts_year_' + str(nfn))
            ts_doy_dict_keys = []
            for nfn, ts_doy_fn in enumerate(ts_doy_fullfns):
                fn_dict['ts_doy_' + str(nfn)] = ts_doy_fn
                ts_doy_dict_keys.append('ts_doy_' + str(nfn))
            ts_dayfrac_dict_keys = []  
            for nfn, ts_dayfrac_fn in enumerate(ts_dayfrac_fullfns):
                fn_dict['ts_dayfrac_' + str(nfn)] = ts_dayfrac_fn
                ts_dayfrac_dict_keys.append('ts_dayfrac_' + str(nfn))

            # ===== PROCESS THE DATA =====
            #Expand extent to include buffered region around glacier polygon
            warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
            #Warp everything to common res/extent/proj
            z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
            z1_res = np.min([z1_gt[1], -z1_gt[5]])
            ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                               t_srs=gf.aea_srs, verbose=False, r='cubic')
            ds_dict = dict(zip(fn_dict.keys(), ds_list))
            gf.ds_dict = ds_dict

            if 'z1' in ds_dict:
                #This is False over glacier polygon surface, True elsewhere - can be applied directly
                glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                glac_geom_mask_copy = glac_geom_mask.copy()
                gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                # Debris cover
                dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)

                # ====== SURFACE TEMPERATURE =====
                find_ts = True
                ts_key_count = 0
                if debug:
                    print('\nSURFACE TEMPERATURE')
                while find_ts and ts_key_count < len(ts_dict_keys):
                    ts_key = ts_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts = False
                        # Export raster
                        ts_fn = rgiid + '_ts.tif'
                        ts_fp = debris_prms.oggm_ts_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_fp):
                            os.makedirs(ts_fp)
                        iolib.writeGTiff(gf.ts, ts_fp + ts_fn, gf.ds_dict['z1'])
                        ts_fn_wglacier = ts_fp + ts_fn
                        
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_fn:', ts_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = malib.calcperc(var_full2plot, (2,98))
                            plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'Ts (degC)', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS YEAR =====
                find_ts_year = True
                ts_key_count = 0
                if debug:
                    print('\nYEAR')
                while find_ts_year and ts_key_count < len(ts_year_dict_keys):
                    ts_key = ts_year_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_year_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_year = False
                        # Export raster
                        ts_year_fn = rgiid + '_ts_year.tif'
                        ts_year_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_year_fp):
                            os.makedirs(ts_year_fp)
                        iolib.writeGTiff(gf.ts, ts_year_fp + ts_year_fn, gf.ds_dict['z1'])
                        ts_year_fn_wglacier = ts_year_fp + ts_year_fn
                        
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_year_fn:', ts_year_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (2013,2018)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_year'], 'inferno', 'Year', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS DOY =====
                find_ts_doy = True
                ts_key_count = 0
                if debug:
                    print('\nDAY OF YEAR')
                while find_ts_doy and ts_key_count < len(ts_doy_dict_keys):
                    ts_key = ts_doy_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_doy_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_doy = False
                        # Export raster
                        ts_doy_fn = rgiid + '_ts_doy.tif'
                        ts_doy_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_doy_fp):
                            os.makedirs(ts_doy_fp)
                        iolib.writeGTiff(gf.ts, ts_doy_fp + ts_doy_fn, gf.ds_dict['z1'])
                        ts_doy_fn_wglacier = ts_doy_fp + ts_doy_fn
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_doy_fn:', ts_doy_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (160,280)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_doy'], 'inferno', 'DOY', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS DAYFRAC =====
                find_ts_dayfrac = True
                ts_key_count = 0
                if debug:
                    print('\nDAY FRACTION')
                while find_ts_dayfrac and ts_key_count < len(ts_dayfrac_dict_keys):
                    ts_key = ts_dayfrac_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_dayfrac_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_dayfrac = False
                        # Export raster
                        ts_dayfrac_fn = rgiid + '_ts_dayfrac.tif'
                        ts_dayfrac_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_dayfrac_fp):
                            os.makedirs(ts_dayfrac_fp)
                        iolib.writeGTiff(gf.ts, ts_dayfrac_fp + ts_dayfrac_fn, gf.ds_dict['z1'])
                        ts_dayfrac_fn_wglacier = ts_dayfrac_fp + ts_dayfrac_fn
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_dayfrac_fn:', ts_dayfrac_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (0,24)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_dayfrac'], 'inferno', 'dayfrac', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                
                        
            # ===== Add layers =====
            # Record filenames if all exist
            if (ts_fn_wglacier is not None and ts_year_fn_wglacier is not None and ts_doy_fn_wglacier is not None and 
                ts_dayfrac_fn_wglacier is not None): 
                glac_idx_ts = ts_fns_df.loc[ts_fns_df['RGIId'] == rgiid, :].index.values
                ts_fns_df.loc[glac_idx_ts,:] = [rgiid, ts_fn_wglacier, ts_year_fn_wglacier, ts_doy_fn_wglacier, 
                                                ts_dayfrac_fn_wglacier]
            else:
                print('No Ts data!', glac_str)
                troubleshoot_fp = (debris_prms.output_fp + 'errors/no_Ts_data/' + debris_prms.roi + '/')
                if not os.path.exists(troubleshoot_fp):
                    os.makedirs(troubleshoot_fp)
                txt_fn_extrapfail = glac_str + "-noTs.txt"
                with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                    text_file.write(glac_str + ' no surface temperature data')
                
    # Save updated filenames
    ts_fns_df.to_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn, index=False)

0 0 RGI60-03.00108
1 1 RGI60-03.00113
2 2 RGI60-03.00123
3 3 RGI60-03.00175
4 4 RGI60-03.00183
5 5 RGI60-03.00237
6 6 RGI60-03.00283
7 7 RGI60-03.00366
No Ts data! 03.00366
8 8 RGI60-03.00419
9 9 RGI60-03.00832
10 10 RGI60-03.01405
11 11 RGI60-03.01408
12 12 RGI60-03.01412
13 13 RGI60-03.01416
14 14 RGI60-03.01422
15 15 RGI60-03.01444
16 16 RGI60-03.01446
17 17 RGI60-03.01447
18 18 RGI60-03.01482
19 19 RGI60-03.01483
20 20 RGI60-03.01493
21 21 RGI60-03.01496
22 22 RGI60-03.01499
23 23 RGI60-03.01501
24 24 RGI60-03.01502
25 25 RGI60-03.01506
26 26 RGI60-03.01509
27 27 RGI60-03.01514
28 28 RGI60-03.01516
29 29 RGI60-03.01517
No Ts data! 03.01517
30 30 RGI60-03.01519
31 31 RGI60-03.01520
32 32 RGI60-03.01522
33 33 RGI60-03.01523
34 34 RGI60-03.01525
35 35 RGI60-03.01527
36 36 RGI60-03.01529
37 37 RGI60-03.01530
38 38 RGI60-03.01533
39 39 RGI60-03.01535
40 40 RGI60-03.01544
41 41 RGI60-03.01555
No Ts data! 03.01555
42 42 RGI60-03.01564
43 43 RGI60-03.01565
44 44 RGI60-03.01566
45 45 RGI60-

In [6]:
print('\nDONE\n')


DONE

